In [1]:
import pytesseract
from PIL import Image
import re

In [2]:
'''
Patient First Name: William
Patient Last Name: Hicks
Rx#: 210280140357
Drug Name: INVOKANA TABS
NDC: 50458-0141-90
Filled date: 09/16/2021
Day supply: 
Quantity: 90
Amount:
Primary Paid Amount:
Copay: $120.00
'''

'\nPatient First Name: William\nPatient Last Name: Hicks\nRx#: 210280140357\nDrug Name: INVOKANA TABS\nNDC: 50458-0141-90\nFilled date: 09/16/2021\nDay supply: \nQuantity: 90\nAmount:\nPrimary Paid Amount:\nCopay: $120.00\n'

In [2]:
import nltk
from nltk.corpus import stopwords
import pytesseract
from PIL import Image
import re
import os

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

stopwords = stopwords.words('english')
path = r"docs"
dir_list = os.listdir(path)


data = dict()
for file in dir_list:
    mx = ''
#     w, h = Image.open(path+'\\'+file).size
#     mx = pytesseract.image_to_string(Image.open(path+'\\'+file).crop((0, 0, w, int(h/2))).convert('L'))
    mx = pytesseract.image_to_string(Image.open(path+'\\'+file).convert('L'))
    mx = mx.replace('\n\n',' ')
    mx = mx.replace('\n',' ')
    res = []
    for x in mx.split():
        st = ' '.join([w for w in x.split() if w not in stopwords])
        res.append(st)
    data[file.split('.')[0]] = ' '.join(res)

In [4]:
data['6']

'Promised: 8/14/21, 10\'24 AM WAITING oieo Resyri” B GR T Gross, Margaret 105 Brown Street Sulte 200, Tecumseh, MI DOB: 953 TEL: (517)902-1370 www. Cvs. comvdruginfo Prescription Information \' ODACTRA 12 SQ-HDM SL | @ TABLET Place 1 tablet   tongue AROAY  daily. Revisit needed  tal - additional refills. EVENING = | IrBReSIANARLEBARSE:  tongue. Do  chew  LG swallow whole. See 3ok  e Mammation | = DO O eat  drink anything  5 minutes  taking  medication. - Wash hands  use - Save time  ReadyFill  we\'ll   refills ready  you. Ask  enrolling today! Receipt & Refill Information ~ CVS Pharmacy BO7 Wast Chicago Biv Tecumseh, Mi 49286 srorers0s | ODACTRA 12 SQ-HDM SL TABLET rx: 1953336 07 QTY: 30 EA INSURANCE INFORMATION: PDP BNE10011 PNCTRXMEDD CAP: Safety 1P 2404 6% ZGWPSIN AUTW 2122632022670369%% MFR PKG: Yes REFILL: 4  11/17/21 MFR: ALK-ABELLO, INC PRSCBR: Georgiana Sanders DAYS SUPPLY: 30 . = DATE FILLED: 8/14/21 RETAIL PRICE:$388.99 | AMOUNT DUE: $158.88 Notes   Pharmacy This medication  

In [3]:
import pickle
with open('data.pkl','wb') as fd:
    pickle.dump(data,fd,pickle.HIGHEST_PROTOCOL)

## Using spacy PhaseMatcher for extracting name:

In [5]:
import spacy
# !python -m spacy download en
nlp = spacy.load('en_core_web_sm')

In [3]:
import spacy
from spacy.matcher import PhraseMatcher
from nltk.corpus import words
import re

from transformers import AutoTokenizer, AutoModelForTokenClassification

from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp_hf = pipeline("ner", model=model, tokenizer=tokenizer)

nlp_1 = spacy.load("en_core_web_sm")
nlp_2 = spacy.load("en_core_web_trf")

eng_dict_words = set(words.words())
eng_dict_words.add('oniine')

def ner_detect_spacy_and_hugging_face(text):                         
    
    
    #spacy
    
    text = text.replace(',','')
    sents = nlp_1(text)

    names_list_1 = []
    
    for entity in sents.ents:
        if entity.label_ == 'PERSON':
            # print('spacy persons',str(entity))
            # check if word in dict, removing last char to account for plural
            if (not all(( ((w in eng_dict_words) or (w.rstrip('s') in eng_dict_words)) for w in re.sub(r'[^\w\s]','',str(entity).lower()).split())) and (not any(char.isdigit() for char in str(entity)))):
                  if 14>max([len(m) for m in str(entity).split()])>3: # length of name limt
                    names_list_1.append(str(entity))
    
    sents = nlp_2(text)
    names_list_2 = []
    
    for entity in sents.ents:
        if entity.label_ == 'PERSON':
            # print('spacy persons',str(entity))
            # check if word in dict, removing last char to account for plural
            if (not all(( ((w in eng_dict_words) or (w.rstrip('s') in eng_dict_words)) for w in re.sub(r'[^\w\s]','',str(entity).lower()).split())) and (not any(char.isdigit() for char in str(entity)))):
              if 14>max([len(m) for m in str(entity).split()])>3: # length of name limt
                    names_list_2.append(str(entity))
                    
    
    # hugging face transformers
    names_list_3= []
    
    list_words = text.split()
    ner_results = nlp_hf(text)
    for i in range(len(ner_results)):
        if ner_results[i]['score']>0.65:
            k= ner_results[i]['entity'] == 'I-PER' or ner_results[i]['entity'] == 'B-PER'
            if k==True:
              names_list_3.append(ner_results[i]['word'])
    
    
    # combine all the lists
    names_list = []
    for indd in range(max(len(names_list_1),len(names_list_2),len(names_list_3))):
        if indd<len(names_list_1):
            names_list.append(names_list_1[indd])
        
        if indd<len(names_list_2):
            names_list.append(names_list_2[indd])
            
        if indd<len(names_list_3):
            names_list.append(names_list_3[indd])
                    
    
    # post processing the lists
           
    found_names =[]
    for name in names_list: # name assumed to be the first two-word string
        if len(name.split())>1:
            found_names.append(name)
    
    if not found_names:
        # regex for all caps words
        matches = re.findall(r"([A-Z]+\s?[A-Z]+[^a-z0-9\W])",text)
        for match in matches:
            if len(match.split())>1:
                if not all(m in eng_dict_words for m in match.lower().split()): # must not be both english words
                    if 14>max([len(m) for m in match.split()])>3: # at leaset one four letter word
                        found_names.append(match)
                        
    # filter some names
    found_names_filtered = found_names.copy()
    for name in found_names:
        for word in name.split():
            if ('tab' in word.lower()) or ('invoice' in word.lower()):
                found_names_filtered.remove(name)
    
    return found_names_filtered



for i in range(1,26):
    if i<10:
        continue
    if(i not in [10,25]):
        #print('i',i)
        names = ner_detect_spacy_and_hugging_face(data[str(i)])
        print(names[0])
    else:
        print('')
        
        
        
        

Georgiana Sanders
